In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import json
import csv

In [2]:
with open('kernels.json') as json_file:
    kernels = json.load(json_file)

In [3]:
with open('squezzenet_layers.txt') as json_file:
    data = json.load(json_file)
    
    names = []
    for layer in data:
        names.append(layer)
        
    itens = []
    for name in names:
        itens.append(data[name])

In [4]:
    conv_2d = [[],[],[],[],[],[],[],[],[],[],[]]
    max_pol = [[],[],[],[],[],[],[],[],[],[],[]]
    
    conv_2d_y = []
    max_pol_y = []
    
    newData = {}
    i = 0
    for kernel,layer in zip(kernels,itens):
        
        if(kernel["kernel"] == "CONV_2D"):
        
            conv_2d[0].extend(np.repeat(layer["output"][0], len(kernel['average_power_per_image'])))
            conv_2d[1].extend(np.repeat(layer["output"][1], len(kernel['average_power_per_image'])))
            conv_2d[2].extend(np.repeat(layer["output"][2], len(kernel['average_power_per_image'])))
            conv_2d[3].extend(np.repeat(layer["filters"][0], len(kernel['average_power_per_image'])))
            conv_2d[4].extend(np.repeat(layer["filters"][1], len(kernel['average_power_per_image'])))
            conv_2d[5].extend(np.repeat(layer["filters"][2], len(kernel['average_power_per_image'])))
            conv_2d[6].extend(np.repeat(layer["stride"], len(kernel['average_power_per_image'])))
            conv_2d[7].extend(np.repeat(layer["input"][0], len(kernel['average_power_per_image'])))
            conv_2d[8].extend(np.repeat(layer["input"][1], len(kernel['average_power_per_image'])))
            conv_2d[9].extend(np.repeat(layer["input"][2], len(kernel['average_power_per_image'])))
            conv_2d[10].extend(np.asarray(kernel['average_power_per_image']))
            i = i + 1
            continue

    training_conv = []
    evaluating_conv = []
    
    conv_2d = np.transpose(conv_2d)
    
    np.random.shuffle(conv_2d)
    
    conv_2d = np.transpose(conv_2d)

    percentage = 0.8
    length = round(percentage * len(conv_2d[0]))
    
    print(len(conv_2d))
    print(len(conv_2d[0]))
    
    for column in conv_2d:
        training_conv.append(column[:length])
        evaluating_conv.append(column[length:])

    print(len(training_conv[0]))
    print(len(evaluating_conv[0]))

    training_y = np.asarray(training_conv[10])
    evaluating_y = np.asarray(evaluating_conv[10])
    
    training_conv = np.asarray(training_conv)
    evaluating_conv = np.asarray(evaluating_conv)
    
    training_conv = training_conv[:-1]
    evaluating_conv = evaluating_conv[:-1]
    
    print(len(training_y))

11
5616
4493
1123
4493


In [5]:
    print(training_conv)

[[ 55.  13.  55. ...  27. 112.  27.]
 [ 55.  13.  55. ...  27. 112.  27.]
 [ 64. 256.  64. ... 128.  96.  48.]
 ...
 [ 55.  13.  55. ...  27. 224.  27.]
 [ 55.  13.  55. ...  27. 224.  27.]
 [ 16.  64.  16. ...  32.   3. 384.]]


In [6]:
    training_conv = np.transpose(training_conv)
    evaluating_conv = np.transpose(evaluating_conv)
    
    reg = LinearRegression()
    reg.fit(training_conv, training_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
    y_pred = reg.predict(evaluating_conv)
    

In [8]:
    print(y_pred)

[3.25586081 3.57959235 3.98634697 ... 3.57959235 3.25586081 3.91823638]


In [9]:
    print('Intercept: \n', reg.intercept_)
    print('Coefficients: \n', reg.coef_)

Intercept: 
 35498033.86323846
Coefficients: 
 [ 7.64547750e+09 -7.64579445e+09 -6.28172416e+09  6.28172416e+09
 -1.10055292e+07  1.10055289e+07 -3.54980292e+07  3.99190810e+07
 -3.96021343e+07 -8.94069672e-04]


In [10]:
    reg.score(evaluating_conv, evaluating_y)

0.7019201933644136

In [11]:
    print('Mean Absolute Error:', metrics.mean_absolute_error(evaluating_y, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(evaluating_y, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(evaluating_y, y_pred)))

Mean Absolute Error: 0.14067104279278433
Mean Squared Error: 0.03874748418606962
Root Mean Squared Error: 0.1968438065727993
